In [2]:
!pip install neo4j

In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [3]:
uri = "neo4j://localhost:7687"  # Replace with your URI 
user = "neo4j"                 # Replace with your username
password = "19091998"          # Replace with your password

driver = GraphDatabase.driver(uri, auth=(user, password))

In [4]:
class Neo4jConnection:
    
    def __init__(self, uri, user, password):
        self.__uri = uri
        self.__user = user
        self.__password = password
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__password))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def execute_query(self, query):
        if self.__driver is not None:
            with self.__driver.session() as session:
                return [record for record in session.run(query)]


In [ ]:
# label fields in the snomed ct graph

q = """ MATCH (n:ObjectConcept)WHERE (n.FSN ENDS WITH '(finding)' or n.FSN ENDS WITH '(disorder)' )
        SET n:SP
        RETURN n LIMIT 10 """

In [5]:
conn = Neo4jConnection(uri, user, password)

query = """ MATCH (c:SP)-[:HAS_DESCRIPTION]->(d:Description)
            WHERE d.descriptionType = 'Synonym'
            RETURN c.id AS ConceptId, c.FSN AS FSN_concept, d.term AS Synonym
"""

results = conn.execute_query(query)

results[0]['Synonym']

'Poisoning by pharmaceutical excipient, NOS'

In [27]:
r_df = pd.DataFrame(results, columns=['ConceptId', 'FSN_concept', 'Synonym'])
r_df.to_csv('SP_Findings_EXT.csv')

In [ ]:
# Run faiss indexing on all the synonyms, find most similar word

In [ ]:
# find node belonging to the synonym

In [18]:
# Find shortest path between concepts 
query = """ MATCH p = shortestPath((start:SP {id: "249473004"})-[*..10]-(end:SP {id: "102608004"}))
            RETURN length(p) as distance """

results = conn.execute_query(query)

for record in results:
    print(record)

/var/folders/fy/m2y3wbmn1lbcwm2rl2s60d_h0000gp/T/ipykernel_75512/1971303142.py:19: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.__driver.session() as session:


<Record distance=3>


In [20]:
query = """ WITH ["249473004","289161009", "223123009", "289163007", "64379006"] AS endNodeIds
            MATCH (start:SP {id: "102608004"}) 
            UNWIND endNodeIds AS endNodeId
            MATCH (end:SP {id: endNodeId})
            CALL {
                WITH start, end
                MATCH p = shortestPath((start)-[*..10]-(end))
                RETURN p
                ORDER BY length(p) ASC LIMIT 1
            }
            RETURN p, length(p) as distance, end.id as endNodeId, start.FSN, end.FSN as NameEndNode """

results = conn.execute_query(query)

results

/var/folders/fy/m2y3wbmn1lbcwm2rl2s60d_h0000gp/T/ipykernel_75512/1971303142.py:19: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.__driver.session() as session:


[<Record p=<Path start=<Node element_id='4:0c414c0f-b769-4d23-ad25-650efdf1893b:99185' labels=frozenset({'Finding', 'SP', 'SPFindings', 'ObjectConcept'}) properties={'FSN': 'Excessive appetite (finding)', 'nodetype': 'concept', 'effectiveTime': '20080731', 'definitionStatusId': '900000000000073002', 'active': '1', 'history': '[{"id": "102608004", "effectiveTime": "20020131", "active": "1", "moduleId": "900000000000207008", "definitionStatusId": "900000000000074008"}]', 'id': '102608004', 'moduleId': '900000000000207008', 'sctid': '102608004'}> end=<Node element_id='4:0c414c0f-b769-4d23-ad25-650efdf1893b:240538' labels=frozenset({'Finding', 'SP', 'SPFindings', 'ObjectConcept'}) properties={'FSN': 'Altered appetite (finding)', 'nodetype': 'concept', 'effectiveTime': '20080731', 'definitionStatusId': '900000000000073002', 'active': '1', 'history': '[{"id": "249473004", "effectiveTime": "20020131", "active": "1", "moduleId": "900000000000207008", "definitionStatusId": "900000000000074008"}

In [ ]:
# extract the shortest path and handle codes with the exact same length 

In [ ]:
# examples of failed matchings in the app 

In [ ]:
conn.close()

In [6]:
# extract the number of descendants and asc for each node 

conn = Neo4jConnection(uri, user, password)

query = """ MATCH (c:SP)-[:HAS_DESCRIPTION]->(d:Description)
WHERE d.descriptionType = 'Synonym'

OPTIONAL MATCH (c)-[:ISA*]->(descendant:SP)
OPTIONAL MATCH (ancestor:SP)-[:ISA*]->(c)
WITH c, d, COLLECT(DISTINCT descendant.id) AS DescendantIDs, COLLECT(DISTINCT ancestor.id) AS AncestorIDs, COLLECT(DISTINCT descendant.FSN) AS DescendantFSN, COLLECT(DISTINCT ancestor.FSN) AS AncestorFSN
RETURN c.id AS ConceptId, c.FSN AS FSN_concept, d.term AS Synonym, SIZE(DescendantIDs) as num_desc,  SIZE(AncestorIDs) as num_anc,  DescendantIDs, AncestorIDs, DescendantFSN, AncestorFSN"""

results = conn.execute_query(query)

results[0]

TransientError: {code: Neo.TransientError.General.MemoryPoolOutOfMemoryError} {message: The allocation of an extra 2.0 MiB would use more than the limit 5.6 GiB. Currently using 5.6 GiB. dbms.memory.transaction.total.max threshold reached}